In [1]:
%load_ext kedro.ipython

[06/27/24 13:29:18] INFO     Registered line magic '%reload_kedro'                                   ]8;id=906494;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=579557;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#58\58]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=788792;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=762238;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#60\60]8;;\

                    INFO     Resolved project path as: /Users/wilsvenleong/Downloads/Learning       ]8;id=722004;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=293182;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#171\171]8;;\
                             Materials/healthhub-content-optimization/content-optimization.                        
                             To set a different path, run '%reload_kedro <project_root>'                           

[06/27/24 13:29:19] INFO     Kedro project Content Optimization                                     ]8;id=515682;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=288557;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#141\141]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=195446;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=348094;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#142\142]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=337081;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=532065;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#148\148]8;;\

In [2]:
import re
import unicodedata

import pandas as pd
from bs4 import BeautifulSoup

In [3]:
class HTMLExtractor:
    """
    A class to extract and process various elements from HTML content
    using BeautifulSoup.

    Attributes:
        soup (BeautifulSoup): A BeautifulSoup object.
    """

    def __init__(self, html_content: str):
        """
        Initializes the HTMLExtractor with the given HTML content.

        Args:
            html_content (str): The HTML content to be processed.
        """
        self.soup = self.preprocess_html(html_content)

    @classmethod
    def clean_text(cls, text: str) -> str:
        """
        Cleans the given text by normalizing Unicode characters,
        handling special symbols, replacing problematic characters,
        and removing multiple whitespace.

        Args:
            text (str): The input text to be cleaned.

        Returns:
            str: The cleaned text.
        """
        # Normalize Unicode characters
        text = unicodedata.normalize("NFKD", text)
        # Use ASCII encoding to handle special symbols e.g. copyright \xa9
        text = text.encode("ascii", "ignore").decode("utf-8")

        # Replace common problematic characters
        text = text.replace("\xa0", " ")  # non-breaking space
        text = text.replace("\u200b", "")  # zero-width space
        text = text.replace("\u2028", "\n")  # line separator
        text = text.replace("\u2029", "\n")  # paragraph separator

        # Replace multiple whitespace with single space
        text = re.sub(r"\s+", " ", text)
        return text.strip()

    @classmethod
    def preprocess_html(cls, html_content: str) -> BeautifulSoup:
        """
        Preprocesses the given HTML content by replacing all <br>
        tags with newline characters.

        Args:
            html_content (str): The HTML content to be preprocessed.

        Returns:
            BeautifulSoup: The preprocessed HTML content as a BeautifulSoup object.
        """
        soup = BeautifulSoup(html_content, "html.parser")

        # Find all <br> tags and replace them with newline
        for br in soup.find_all("br"):
            br.replace_with("\n")

        return soup

    def extract_related_sections(self) -> list[str]:
        """
        Extracts "Related:" sections and "Read these next:" items from the HTML content.

        Returns:
            list[str]: A list of related sections and "Read these next:" items.
        """
        related_sections = []
        read_these_next_ul = None
        # Extract "Related:" sections and "Read these next:" items
        for tag in self.soup.find_all(["p", "ul"]):
            if tag.name == "p" and tag.find("strong"):
                if "Related:" in tag.text:
                    related_sections.append(
                        re.sub(r"Related: ", "", self.clean_text(tag.text))
                    )
                elif "Read these next:" in tag.text:
                    read_these_next_ul = tag.find_next_sibling("ul")
            elif tag == read_these_next_ul:
                for li in tag.find_all("li"):
                    related_sections.append(self.clean_text(li.text))

        return related_sections

    def extract_text(self) -> str:
        """
        Extracts the main content from the HTML content.

        Returns:
            str: The main content extracted from the HTML content.

        Note:
            This function unwraps the HTML content if it is contained in a <div>. It then extracts the
            main content by iterating over the tags in the soup. The following tags are considered:

                - h1, h2, h3, h4, h5, h6: These tags are treated as key headers and are paragraphed between them.
                - p: This tag is treated as a paragraph. <em> tags are removed from the text.
                    * If the text does not contain sentences about HealthHub app, Google Play, or Apple Store,
                    and it contains a strong tag, it is treated differently based on the text content.
                - ul: This tag is treated as an unordered list. If it is the child of a <div>, it is treated as a list.
                - ol: This tag is treated as an ordered list.
                - div: This tag is treated as a text within a div.

            The extracted content is stored in a list and then processed. Double newlines are replaced with single
            newlines and whitespace is stripped. If the processed text is empty, the function attempts to extract the
            content from the <div> tags.
        """
        # Unwrap if the HTML content is contained in a div
        if self.soup.div is not None:
            self.soup.div.unwrap()

        # Extract the main content
        content = []
        for tag in self.soup.find_all(
            ["h1", "h2", "h3", "h4", "h5", "h6", "div", "p", "ul", "ol"]
        ):
            if tag.name in ["h1", "h2", "h3", "h4", "h5", "h6"]:
                # Provide paragraphing between key headers
                content.append("\n")
                content.append(self.clean_text(tag.text))

            elif tag.name == "p":
                # Remove all em tags
                for em in tag.find_all("em"):
                    em.extract()
                # Get the remaining text
                text = tag.get_text()
                # Remove sentences about HealthHub app, Google Play, and Apple Store
                if not re.search(
                    r"(HealthHub app|Google Play|Apple Store|Parent Hub|References)",
                    text,
                ):
                    if tag.find("strong"):
                        if "Related:" in tag.text:
                            text = self.clean_text(tag.text)
                            content.append(re.sub(r"\n", " ", text))
                        elif "Read these next:" in tag.text:
                            content.append(self.clean_text(tag.text))
                    else:
                        content.append(self.clean_text(text))
            # For unordered lists
            elif (
                tag.name == "ul" and tag.parent.name == "div"
            ):  # not "ul" so we avoid duplicates
                for li in tag.find_all("li"):
                    content.append("- " + self.clean_text(li.text))
            # For ordered lists
            elif tag.name == "ol":
                for i, li in enumerate(tag.find_all("li")):
                    content.append(f"{i + 1}. " + self.clean_text(li.text))
            # For texts within div
            elif tag.name == "div":
                content.append(self.clean_text(tag.text))

            content.append("")  # Add a blank line after each element

        # Remove empty strings from content
        content = [c for c in content if c]

        # Replace double newlines with single newlines and strip whitespace
        processed_text = "\n".join(content).replace("\n\n", "\n").strip()

        # Edge case - HTML content contained in div tags
        if processed_text.strip() == "":
            content = []
            # Unwrap if the HTML content is contained in a div
            if self.soup.div is not None:
                self.soup.div.unwrap()
                # For texts within div
                for tag in self.soup.find_all("div"):
                    if tag.name == "div":
                        content.append(self.clean_text(tag.text))

                # Replace double newlines with single newlines and strip whitespace
                processed_text = "\n".join(content).replace("\n\n", "\n").strip()

        return processed_text

    def extract_links(self) -> list[tuple[str, str]]:
        """
        Extracts the title and URL from all the anchor tags in the HTML content.

        Returns:
            list[tuple[str, str]]:
                A list of tuples containing the title and URL of each anchor tag.
        """
        url_records = []

        # Extract title/text and links from anchor tags
        for link in self.soup.find_all("a"):
            url = link.get("href")
            # Ignore footnotes
            if url != "#footnotes":
                text = link.get("title") or link.get_text()
                record = text, url
                url_records.append(record)

        return url_records

    def extract_headers(self) -> list[tuple[str, str]]:
        """
        Extracts the headers from the HTML content.

        Returns:
            list[tuple[str, str]]:
                A list of tuples containing the text and tag name of
                each header found in the HTML content.
        """
        headers = []

        for title in self.soup.find_all(["h1", "h2", "h3", "h4", "h5", "h6"]):
            tag = title.name
            text = title.get_text()
            # Ignore References
            if text != "References":
                record = text, tag
                headers.append(record)

        return headers

In [4]:
# ruff: noqa: F821
catalog.list()


[
    'all_contents',
    'all_contents_processed',
    'all_contents_extracted',
    'all_extracted_text',
    'parameters',
    'params:columns_to_keep',
    'params:columns_to_keep.cost-and-financing',
    'params:columns_to_keep.diseases-and-conditions',
    'params:columns_to_keep.health-statistics',
    'params:columns_to_keep.live-healthy-articles',
    'params:columns_to_keep.medical-care-and-facilities',
    'params:columns_to_keep.medications',
    'params:columns_to_keep.program-sub-pages',
    'params:columns_to_keep.programs',
    'params:columns_to_keep.support-group-and-others',
    'params:metadata',
    'params:metadata.cost-and-financing',
    'params:metadata.cost-and-financing.uuid',
    'params:metadata.cost-and-financing.content_title',
    'params:metadata.cost-and-financing.content_body',
    'params:metadata.diseases-and-conditions',
    'params:metadata.diseases-and-conditions.uuid',
    'params:metadata.diseases-and-conditions.content_title',
    'params:met

### Load all the contents

In [5]:
# ruff: noqa: F821
all_contents = catalog.load("all_contents")
all_contents

                    INFO     Loading data from all_contents (PartitionedDataset)...             ]8;id=310634;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=74897;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/io/data_catalog.py#508\508]8;;\


{
    'export-published-cost-and-financing_14062024_data': <bound method AbstractVersionedDataset.load of <kedro_datasets.pandas.excel_dataset.ExcelDataset object at 0x1181a8c50>>,
    'export-published-diseases-and-conditions_13062024_data': <bound method AbstractVersionedDataset.load of <kedro_datasets.pandas.excel_dataset.ExcelDataset object at 0x169ad8c20>>,
    'export-published-health-statistics_14062024_data': <bound method AbstractVersionedDataset.load of <kedro_datasets.pandas.excel_dataset.ExcelDataset object at 0x1699e0530>>,
    'export-published-live-healthy-articles_14062024_data': <bound method AbstractVersionedDataset.load of <kedro_datasets.pandas.excel_dataset.ExcelDataset object at 0x169aee330>>,
    'export-published-medical-care-and-facilities_14062024_data': <bound method AbstractVersionedDataset.load of <kedro_datasets.pandas.excel_dataset.ExcelDataset object at 0x169aeeb10>>,
    'export-published-medications_14062024_data': <bound method AbstractVersionedDatas

### Load the content category

In [6]:
content_category = "live-healthy-articles"

### Use `content_category` as key to access the corresponding content

In [7]:
df = all_contents[f"export-published-{content_category}_14062024_data"]()
# Drop all columns which have only null values
df = df.dropna(axis=1, how="all")
print(df.shape)

(1155, 60)


### View all columns to keep for corresponding content category

In [8]:
# ruff: noqa: F821
columns_to_keep = catalog.load("params:columns_to_keep")
columns_to_keep

[06/27/24 13:29:25] INFO     Loading data from params:columns_to_keep (MemoryDataset)...        ]8;id=361730;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=890586;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/io/data_catalog.py#508\508]8;;\


{
    'cost-and-financing': [
        'id',
        'Content.Name',
        'CostAndFinancing_Title',
        'CostAndFinancing_ArticleCatNames',
        'CostAndFinancing_CoverImgUrl',
        'CostAndFinancing_FullUrl',
        'CostAndFinancing_FullUrl2',
        'CostAndFinancing_FriendlyUrl',
        'CostAndFinancing_CategoryDesc',
        'CostAndFinancing_ContentBody',
        'CostAndFinancing_ENKeywords',
        'CostAndFinancing_FeatureTitle',
        'CostAndFinancing_PRName',
        'CostAndFinancing_AlternateImageText',
        'CostAndFinancing_DateModified',
        'CostAndFinancing_NumberofViews',
        'CostAndFinancing_LastMonthViewCount',
        'CostAndFinancing_LastTwoMonthsView',
        'Page Views',
        'Engagement Rate',
        'Bounce Rate',
        'Exit Rate',
        'Scroll %',
        '% of Total Views',
        'Cumulative % of Total Views'
    ],
    'diseases-and-conditions': [
        'id',
        'Content.Name',
        'DiseasesConditi

### Get columns to drop corresponding to `content_category`

In [9]:
relevant_columns = columns_to_keep[content_category]
relevant_columns


[
    'id',
    'Content.Name',
    'LiveHealthyArticle_Title',
    'LiveHealthyArticle_ArticleCateName',
    'LiveHealthyArticle_CoverImgUrl',
    'LiveHealthyArticle_FullUrl',
    'LiveHealthyArticle_FullUrl2',
    'LiveHealthyArticle_FriendlyUrl',
    'LiveHealthyArticle_CategoryDes',
    'LiveHealthyArticle_Content_Body',
    'LiveHealthyArticle_ENKeywords',
    'LiveHealthyArticle_FeatureTitle',
    'LiveHealthyArticle_PRName',
    'LiveHealthyArticle_AlternateImageText',
    'LiveHealthyArticle_DateModified',
    'LiveHealthyArticle_Number_of_View',
    'LiveHealthyArticle_Lastmonthview',
    'LiveHealthyArticle_LastTwoMonthsView',
    'Page Views',
    'Engagement Rate',
    'Bounce Rate',
    'Exit Rate',
    'Scroll %',
    '% of Total Views',
    'Cumulative % of Total Views'
]

In [10]:
# Keep all relevant columns
df = df[relevant_columns]
print(df.shape)

(1155, 25)


In [11]:
# Remaining columns
df.columns


Index(['id', 'Content.Name', 'LiveHealthyArticle_Title',
       'LiveHealthyArticle_ArticleCateName', 'LiveHealthyArticle_CoverImgUrl',
       'LiveHealthyArticle_FullUrl', 'LiveHealthyArticle_FullUrl2',
       'LiveHealthyArticle_FriendlyUrl', 'LiveHealthyArticle_CategoryDes',
       'LiveHealthyArticle_Content_Body', 'LiveHealthyArticle_ENKeywords',
       'LiveHealthyArticle_FeatureTitle', 'LiveHealthyArticle_PRName',
       'LiveHealthyArticle_AlternateImageText',
       'LiveHealthyArticle_DateModified', 'LiveHealthyArticle_Number_of_View',
       'LiveHealthyArticle_Lastmonthview',
       'LiveHealthyArticle_LastTwoMonthsView', 'Page Views', 'Engagement Rate',
       'Bounce Rate', 'Exit Rate', 'Scroll %', '% of Total Views',
       'Cumulative % of Total Views'],
      dtype='object')

### Get metadata for corresponding content category

In [12]:
# ruff: noqa: F821
metadata = catalog.load("params:metadata")
metadata

[06/27/24 13:29:26] INFO     Loading data from params:metadata (MemoryDataset)...               ]8;id=990158;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=990898;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/io/data_catalog.py#508\508]8;;\


{
    'cost-and-financing': {
        'uuid': 'id',
        'content_title': 'CostAndFinancing_Title',
        'content_body': 'CostAndFinancing_ContentBody'
    },
    'diseases-and-conditions': {
        'uuid': 'id',
        'content_title': 'DiseasesCondition_Title',
        'content_body': 'DiseasesCondition_ContentBody'
    },
    'health-statistics': {
        'uuid': 'id',
        'content_title': 'HealthStatistics_Title',
        'content_body': 'HealthStatistics_ContentBody'
    },
    'live-healthy-articles': {
        'uuid': 'id',
        'content_title': 'LiveHealthyArticle_Title',
        'content_body': 'LiveHealthyArticle_Content_Body'
    },
    'medical-care-and-facilities': {
        'uuid': 'id',
        'content_title': 'MedicalCareFaci_Title',
        'content_body': 'MedicalCareFaci_ContentBody'
    },
    'medications': {
        'uuid': 'id',
        'content_title': 'Medication_Title',
        'content_body': 'Medication_ContentBody'
    },
    'program-sub-

In [13]:
content_body = metadata[content_category]["content_body"]
content_body

'LiveHealthyArticle_Content_Body'

### Flag all articles with no content

In [14]:
# Mark articles with no content `to_remove`
df["to_remove"] = df[content_body].apply(
    lambda x: (
        False if pd.notna(x) and re.search(r"(<[div|p|h2].*?>)", str(x)) else True
    )
)
print(df.shape)

(1155, 26)


### Sample 5 articles

In [15]:
tmp = df.sample(5, random_state=42).reset_index(drop=True)
tmp

,id,Content.Name,LiveHealthyArticle_Title,LiveHealthyArticle_ArticleCateName,LiveHealthyArticle_CoverImgUrl,LiveHealthyArticle_FullUrl,LiveHealthyArticle_FullUrl2,LiveHealthyArticle_FriendlyUrl,LiveHealthyArticle_CategoryDes,LiveHealthyArticle_Content_Body,...,LiveHealthyArticle_Lastmonthview,LiveHealthyArticle_LastTwoMonthsView,Page Views,Engagement Rate,Bounce Rate,Exit Rate,Scroll %,% of Total Views,Cumulative % of Total Views,to_remove
0,1445429,Know Your Alcohol Limit: Don’t Be a Party Pooper!,Know Your Alcohol Limit: Don’t Be a Party Pooper!,"Sexual Health and Relationships,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/dont_be_...,www.healthhub.sg/live-healthy/dont_be_a_party_...,dont_be_a_party_pooper,Everyone enjoys a night of partying. But what ...,"<div class=""ExternalClassFF709E64E1BD479883D08...",...,62.0,95.0,224,0.970954,0.029046,0.166023,0.266741,0.000189,0.986617,False
1,1442458,Help Your Child Untangle From The Web,Help Your Child Untangle From The Web,"Mind and Balance,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/help-you...,www.healthhub.sg/live-healthy/help-your-child-...,help-your-child-untangle-from-the-web,Does your child spend long hours in front of t...,"<div class=""ExternalClassF5D63B3B546A42419922B...",...,171.0,291.0,1140,0.807725,0.192275,0.318612,0.337500,0.000963,0.677177,False
2,1444585,Makan Matters: What‘s a Balanced Diet?,Makan Matters: What‘s a Balanced Diet?,"Food and Nutrition,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/makan-ma...,www.healthhub.sg/live-healthy/makan-matters-wh...,makan-matters-whats-a-healthy-diet,Trying to eat better? Find out what makes a he...,"<div class=""ExternalClassB17BB24F34014C7AAEC3F...",...,0.0,0.0,2434,0.702500,0.297500,0.701918,0.368735,0.002057,0.462499,False
3,1443659,Preparing for Pregnancy: 3 Things to Do Now Th...,Preparing for Pregnancy: 3 Things to Do Now Th...,"Body Care,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/3-things...,www.healthhub.sg/live-healthy/3-things-to-do-n...,3-things-to-do-now-that-youre-pregnant,You've stopped eating raw food and you've also...,"<div class=""ExternalClassA116BA2F83BB43CE9D25E...",...,30.0,41.0,1027,0.710130,0.289870,0.706122,0.346641,0.000868,0.707952,False
4,1444177,Time to Exercise: Aeroplane (in Four-Point Kne...,Time to Exercise: Aeroplane (in Four-Point Kne...,"Exercise & Fitness,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/time-to-...,www.healthhub.sg/live-healthy/time-to-exercise...,time-to-exercise-aeroplane-in-four-point-kneeling,"Hey there mum-to-be, let's do some exercise! G...","<div class=""ExternalClass5A08DDBEC9704AB088788...",...,23.0,23.0,277,0.889251,0.110749,0.298611,0.268051,0.000234,0.961391,False


In [16]:
content_title = metadata[content_category]["content_title"]
content_title

'LiveHealthyArticle_Title'

In [17]:
tmp["related_sections"] = None
tmp["extracted_content_body"] = None
tmp["extracted_links"] = None
tmp["extracted_headers"] = None

for index, row in tmp.iterrows():
    # Skip extraction for those articles flagged for removal
    if row["to_remove"]:
        continue

    # Replace all forward slashes with hyphens to avoid saving as folders
    title = row[content_title]
    print(title)

    # Get the HTML content
    html_content = row[content_body]

    # Extract text from HTML using the HTMLExtractor Class
    extractor = HTMLExtractor(html_content)
    related_sections = extractor.extract_related_sections()
    extracted_content_body = extractor.extract_text()
    extracted_links = extractor.extract_links()
    extracted_headers = extractor.extract_headers()

    # Store extracted data into the dataframe
    tmp.at[index, "related_sections"] = related_sections
    tmp.at[index, "extracted_content_body"] = extracted_content_body
    tmp.at[index, "extracted_links"] = extracted_links
    tmp.at[index, "extracted_headers"] = extracted_headers

    # If `extracted_content_body` is empty, we update flag to remove
    if extracted_content_body == "":
        tmp.at[index, "to_remove"] = True

Know Your Alcohol Limit: Don’t Be a Party Pooper!
Help Your Child Untangle From The Web
Makan Matters: What‘s a Balanced Diet?
Preparing for Pregnancy: 3 Things to Do Now That You're Pregnant
Time to Exercise: Aeroplane (in Four-Point Kneeling)


In [18]:
index = 1

tmp.iloc[index]["LiveHealthyArticle_FullUrl"]

'https://www.healthhub.sg/live-healthy/help-your-child-untangle-from-the-web'

In [19]:
tmp.iloc[index]["related_sections"]


[
    'Screen Time',
    'Disconnect To Reconnect Why A Social Media Detox Might Be Good For You',
    '9 Health Hazards of Electronic Devices for Kids',
    'Dealing with Cyber-Bullies',
    'It Is Best Your Child Hears It From You',
    'Dealing With Cyber Bullies',
    'Parent Hub',
    'Happy and Lasting Relationships with Children',
    'Stuck in the Web'
]

In [20]:
tmp.iloc[index]["extracted_content_body"]

'Singapore teens are among the most Internet-savvy in the world. Statistics from the Infocomm Development Authority of Singapore show that 99% of teenagers used the Internet in 2014. Their favourite mobile activities are social networking on platforms like Instagram, instant messaging via apps like Snapchat, and sending and receiving emails.\nOnline activities are enjoyable, so it is easy to spend long hours on the computer. It is normal for your child to want to spend more time on activities that fascinate them, and these interests are often good outlets for learning, creativity and self-expression.\nBut when any activity becomes the major focus of their life to the point where it becomes harmful physically, mentally or socially your child may have a cyber addiction problem. A 2010 study[1] led by the National Institute of Education and the Media Development Authority found that Singaporean youth spend more time than American adolescents on video gaming averaging 20 hours a week, comp

In [21]:
tmp.iloc[index]["extracted_links"]


[
    (
        'Screen Time',
        'http://www.babybonus.msf.gov.sg/parentingresources/web/Young-Children/YoungChildrenPlay_and_Learning/Screen_Time/Young_Children_Screen_Time'
    ),
    ('these ideas', 'https://www.healthhub.sg/live-healthy/ideas-for-an-active-weekend'),
    (
        'Disconnect To Reconnect — Why A Social Media Detox Might Be Good For You',
        'https://www.healthhub.sg/live-healthy/disconnect-to-reconnect-why-a-social-media-detox-might-be-good-for-you'
    ),
    (
        '9 Health Hazards of Electronic Devices for Kids',
        'https://www.healthhub.sg/live-healthy/9-health-hazards-of-electronic-devices-for-kids'
    ),
    (
        'Dealing with Cyber-Bullies',
        'https://www.healthhub.sg/live-healthy/dealing_with_cyber-bullies'
    ),
    (
        'It Is Best Your Child Hears It From You',
        'https://www.healthhub.sg/live-healthy/best_your_child_hears_it_from_you'
    ),
    (
        'Visit Parent Hub, for more useful tips and guides 

In [22]:
tmp.iloc[index]["extracted_headers"]


[
    ('1. Practise What You Preach', 'h2'),
    ('2. Have Control', 'h2'),
    ('3. Be Understanding', 'h2'),
    ('4. Be Alert to Danger', 'h2'),
    ('5. Set Realistic Rules', 'h2')
]